get ndvi

In [ ]:
import pandas as pd

# Define the first few metadata column names
metadata_columns = ['Product_Name', 'MODIS_Product', 'Date', 'Coordinates', 'Identifier']

# Automatically generate reflectance column names from Reflectance_1 to Reflectance_N (up to column KH)
# We already have 5 columns for metadata, so we generate 67 additional columns (from column 6 to 72)
reflectance_columns = [f'Reflectance_{i}' for i in range(1, 290)]  # Adjust this range if necessary

# Combine metadata and reflectance columns
columns = metadata_columns + reflectance_columns

# Load the dataset, assuming it doesn't have a header
data = pd.read_csv(r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\filtered_scaled_Nadir_Reflectance_Band2.csv", header=None, names=columns)

# Replace 'F' with NaN to handle missing/bad data
data.replace('F', pd.NA, inplace=True)
output_file = r'C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\band2.csv'  # Update this path

# Save the updated dataset with column names to the specified CSV file location
data.to_csv(output_file, index=False)


# Show the first few rows of the dataset to verify column names
print(data)



get nirv

In [ ]:
import pandas as pd

# Load the processed Band 1 and Band 2 CSV files
# Replace these file paths with the actual processed files
band1_data = pd.read_csv(r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\band1.csv")  # Band 1 (Red) file path
band2_data = pd.read_csv(r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\band2.csv")  # Band 2 (NIR) file path

# Ensure both dataframes have the same structure and are aligned properly

# Extract the first five columns (metadata)
metadata_columns = ['Product_Name', 'MODIS_Product', 'Date', 'Coordinates', 'Identifier']
metadata = band1_data[metadata_columns]

# Extract reflectance values (assumed to start from column 6 onwards)
red_band = band1_data.iloc[:, 5:].astype(float)  # Nadir Reflectance Band 1 (Red)
nir_band = band2_data.iloc[:, 5:].astype(float)  # Nadir Reflectance Band 2 (NIR)

# Calculate NDVI: (NIR - Red) / (NIR + Red)
ndvi = (nir_band - red_band) / (nir_band + red_band)

# Calculate NIRv: NDVI * NIR
nirv = ndvi * nir_band

# Create a DataFrame for NDVI and NIRv, keeping the metadata
final_dataset = pd.concat([metadata, ndvi.add_prefix('NDVI_'), nirv.add_prefix('NIRv_')], axis=1)

# Specify the file path to save the final dataset
output_file = r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\nirv.csv"  # Update the path

# Save the final dataset with NDVI, NIRv, and the first five columns (metadata)
final_dataset.to_csv(output_file, index=False)

# Display the first few rows of the final dataset
print(final_dataset.head())

# Notify the user about the file save location
print(f"File saved as {output_file}")

get all neccessary data together

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Load DE-Kli data
dekli_data_file = r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\FLX_DE-Kli_FLUXNET2015_SUBSET_2004-2014_1-4\FLX_DE-Kli_FLUXNET2015_SUBSET_DD_2004-2014_1-4.csv"  # Replace with your DE-Kli data file path
dekli_data = pd.read_csv(dekli_data_file)

# Load NDVI and NIRv reflectance data
ndvi_nirv_data_file = r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\nirv.csv"  # Path to your NDVI/NIRv reflectance data file
ndvi_nirv_data = pd.read_csv(ndvi_nirv_data_file)

# Convert AYDDD date format to YYYYMMDD in NDVI/NIRv data
def convert_ayyyyddd_to_yyyymmdd(ayyyyddd):
    year = int(ayyyyddd[1:5])
    day_of_year = int(ayyyyddd[5:])
    date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)
    return date.strftime('%Y%m%d')  # Convert to YYYYMMDD format

# Apply conversion to the NDVI/NIRv DataFrame
ndvi_nirv_data['Timestamp'] = ndvi_nirv_data['Date'].apply(convert_ayyyyddd_to_yyyymmdd)

# Rename NDVI and NIRv reflectance columns
ndvi_columns = [f'NDVI_Reflectance_{m}' for m in range(1,2)]  # m from 1 to 289
nirv_columns = [f'NIRv_Reflectance_{m}' for m in range(1, 2)]  # m from 1 to 289

# Assuming NDVI and NIRv columns are named like 'NDVI_Reflectance_1', 'NDVI_Reflectance_2', ..., adjust this step if needed
original_ndvi_columns = [col for col in ndvi_nirv_data.columns if 'NDVI_Reflectance' in col]
original_nirv_columns = [col for col in ndvi_nirv_data.columns if 'NIRv_Reflectance' in col]

# Rename NDVI columns
ndvi_nirv_data.rename(columns=dict(zip(original_ndvi_columns, ndvi_columns)), inplace=True)

# Rename NIRv columns
ndvi_nirv_data.rename(columns=dict(zip(original_nirv_columns, nirv_columns)), inplace=True)

# Ensure DE-Kli data's TIMESTAMP is a string to match
dekli_data['Timestamp'] = dekli_data['TIMESTAMP'].astype(str)  # Adjust if the column name is different

# Merge DE-Kli data with NDVI and NIRv data on the timestamp
merged_data = pd.merge(dekli_data, ndvi_nirv_data, on='Timestamp', how='inner')

# Example of selecting relevant columns
selected_columns = [
    'TIMESTAMP',  # DE-Kli timestamp
    'TA_F',       # Air temperature
    'P_F',        # Precipitation
    'SW_IN_F',    # Shortwave radiation (or NETRAD if necessary)
    'NEE_VUT_REF',  # Carbon flux (Net Ecosystem Exchange)
    'VPD_F',# Vapor pressure deficit
    'WS_F',
    'NETRAD',
    'G_F_MDS'
]

# Add the renamed NDVI and NIRv reflectance columns to the selected columns
selected_columns.extend(ndvi_columns + nirv_columns)

# Create the final dataset with selected columns
final_dataset = merged_data[selected_columns]

# Specify the file path to save the final dataset
output_file = r"C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\final.csv"# Adjust as necessary

# Save the final dataset to a new CSV file
final_dataset.to_csv(output_file, index=False)

# Display the first few rows of the final dataset
print(final_dataset.head())

# Notify the user about the file save location
print(f"Final dataset saved as {output_file}")


calculate et

In [ ]:
import pandas as pd
import numpy as np

# Load the merged dataset with constants
data_file = r'C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\final.csv'  # Adjust path as needed
data = pd.read_csv(data_file)

# Constants
gamma = 0.066  # Psychrometric constant (kPa/°C)
Cp = 1.013 * 10**-3  # Specific heat of air (MJ/kg°C)
rho = 1.225  # Air density (kg/m³)

# Calculate Aerodynamic Conductance (Ga)
z = 2.0  # Height of measurement (m)
z0 = 0.1  # Roughness length for grassland (m)
data['Ga'] = data['WS_F'] / np.log(z / z0)  # Wind speed in m/s

# Calculate Saturation Vapor Pressure (e_s) and Slope (Δ)
data['e_s'] = 6.11 * np.exp((7.5 * data['TA_F']) / (data['TA_F'] + 237.3))  # Saturation vapor pressure (kPa)
data['Delta'] = (4098 * data['e_s']) / ((data['TA_F'] + 237.3) ** 2)  # Slope of vapor pressure curve (kPa/°C)

# Calculate Surface Conductance (Gs)
g0 = 0.1  # Minimum conductance (mol/m²/s), empirical value
g1 = 0.5  # Sensitivity of conductance to vapor pressure deficit
D_min = 0.1  # Minimum vapor pressure deficit (kPa)

# Calculate Gs based on VPD and NDVI
data['Gs'] = g0 + g1 * (data['VPD_F'] / D_min) * (data['NDVI'] / data['NDVI'].max())

# Calculate ET using the Penman-Monteith equation
# Assuming G (soil heat flux) is zero or from G_F_MDS column
G = data['G_F_MDS'].fillna(0)  # Fill with 0 if not available

# Calculate Vapor Pressure Deficit (D)
data['D'] = data['VPD_F']  # Use the VPD_F column

# Calculate Evapotranspiration (ET)
data['ET'] = ((data['NETRAD'] - G) * data['Delta'] + rho * Cp * data['D'] * data['Ga']) / (data['Delta'] + gamma * (1 + data['Ga'] / data['Gs']))

# Convert ET to mm/day
data['ET_mm_per_day'] = data['ET'] / 2.45 * 86400  # Convert W/m² to mm/day

# Display the first few rows with ET
print(data[['TIMESTAMP', 'ET', 'ET_mm_per_day']].head())

# Save the updated DataFrame with ET to a new CSV file
output_file = r'C:\Users\melio\OneDrive\Desktop\labs and lectures\Seminar\Datasets\final_dataset_with_et.csv'  # Adjust path as needed
data.to_csv(output_file, index=False)

# Notify the user about the file save location
print(f"Updated final dataset with ET saved as {output_file}")
